Modify this cell later

In [1]:
import arcpy
import arcview
from arcpy import env
import os
import gdal
import shutil
import numpy as np
import pandas as pd
import shapefile
import matplotlib.pyplot as plt
import datetime

%run D:\Python_Functions\FolderCreater.py

C:\Users\grui9\anaconda3\envs\TSEB\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Product and extension licensing is no longer handled with this method.
  


In [2]:
def reshape_df_statistics(array):
    '''
    put an array in and then get the statistics as the output
    '''
    array = array.reshape(-1)
    df = pd.DataFrame()
    df['statistics'] = array
    df_describe = df.describe()
    statistics = df_describe['statistics'].values
    number = len(statistics)
    out_list = [0] * number
    for inumber in range(number):
        out_list[inumber] = statistics[inumber]
    return(out_list)    

In [3]:
LAI_Grids = r'D:\Project_Wellsville\10_LAI_Corn_Alfalfa\1_ArcGIS_Pro_Coordinates\Grid_072X.shp'
shp_info = shapefile.Reader(LAI_Grids)
records = shp_info.records()
num_records = len(records)
print("Totally",num_records,"ground LAI measurements.")

Totally 213 ground LAI measurements.


In [4]:
# information about the shapefile
dir_tmp = r'D:\Project_Wellsville\10_LAI_Corn_Alfalfa\3_Model_Test_Folder_072X'
FolderCreater(dir_tmp)
arcpy.env.overwriteOutput = True
Shp_Grid_FileName = "FocusOnThisGrid.shp"
noDataValue = np.nan

# information about image. This image is resampled one (10 cm for my case)
dir_image = r'D:\Project_Wellsville\4_LAI_VI_Relationship\2_Well_20200720\20200720_BGRRENIRTr_Clip.tif'
# tell the band number
band_B = 0; band_G = 1; band_R = 2; band_RE = 3; band_NIR = 4; band_Tr = 5

In [ ]:
start = datetime.datetime.now()
ID = []
B_mean = [];B_std = [];B_min = [];B_25 = [];B_50 = [];B_75 = [];B_max = [];
G_mean = [];G_std = [];G_min = [];G_25 = [];G_50 = [];G_75 = [];G_max = [];
R_mean = [];R_std = [];R_min = [];R_25 = [];R_50 = [];R_75 = [];R_max = [];
RE_mean = [];RE_std = [];RE_min = [];RE_25 = [];RE_50 = [];RE_75 = [];RE_max = [];
NIR_mean = [];NIR_std = [];NIR_min = [];NIR_25 = [];NIR_50 = [];NIR_75 = [];NIR_max = [];
T_mean = [];T_std = [];T_min = [];T_25 = [];T_50 = [];T_75 = [];T_max = [];
NDVI_mean = [];NDVI_std = [];NDVI_min = [];NDVI_25 = [];NDVI_50 = [];NDVI_75 = [];NDVI_max = [];
GNDVI_mean = [];GNDVI_std = [];GNDVI_min = [];GNDVI_25 = [];GNDVI_50 = [];GNDVI_75 = [];GNDVI_max = [];
RENDVI_mean = [];RENDVI_std = [];RENDVI_min = [];RENDVI_25 = [];RENDVI_50 = [];RENDVI_75 = [];RENDVI_max = [];

for ishp in range(num_records):
    env.workspace = LAI_Grids
    Shapefile = LAI_Grids.replace('.shp', '_lyr')
    arcpy.MakeFeatureLayer_management(LAI_Grids, Shapefile)
    arcpy.SelectLayerByAttribute_management(Shapefile, 'NEW_SELECTION', '"FID" =' + str(ishp))
    arcpy.CopyFeatures_management(Shapefile,
                                  dir_tmp + "\\" + Shp_Grid_FileName)
    
    # read the matrix for each location
    out_raster = arcpy.gp.ExtractByMask_sa(dir_image, dir_tmp + "\\" + Shp_Grid_FileName, 
                                           dir_tmp+"\\"+"GridID_"+str(ishp)+".tif")
    array_grid = arcpy.RasterToNumPyArray(dir_tmp+"\\"+"GridID_"+str(ishp)+".tif", 
                                          nodata_to_value=noDataValue)
    # get each band
    array_B = array_grid[band_B,:,:]
    array_G = array_grid[band_G,:,:]
    array_R = array_grid[band_R,:,:]
    array_RE = array_grid[band_RE,:,:]
    array_NIR = array_grid[band_NIR,:,:]
    array_Tr = array_grid[band_Tr,:,:]/100

    # calculate the NDVI
    array_ndvi = (array_grid[band_NIR,:,:]-array_grid[band_R,:,:])/(array_grid[band_NIR,:,:]+array_grid[band_R,:,:])
    array_gndvi = (array_grid[band_NIR,:,:]-array_grid[band_G,:,:])/(array_grid[band_NIR,:,:]+array_grid[band_G,:,:])
    array_rendvi = (array_grid[band_NIR,:,:]-array_grid[band_RE,:,:])/(array_grid[band_NIR,:,:]+array_grid[band_RE,:,:])
    
    sta_B = reshape_df_statistics(array_B)
    sta_G = reshape_df_statistics(array_G)
    sta_R = reshape_df_statistics(array_R)
    sta_RE = reshape_df_statistics(array_RE)
    sta_NIR = reshape_df_statistics(array_NIR)
    sta_Tr = reshape_df_statistics(array_Tr)
    sta_NDVI = reshape_df_statistics(array_ndvi)
    sta_GNDVI = reshape_df_statistics(array_gndvi)
    sta_RENDVI = reshape_df_statistics(array_rendvi)

    # get the average for each band
    ID.append(ishp+1)
    B_mean.append(sta_B[1]);B_std.append(sta_B[2]);B_min.append(sta_B[3]);B_25.append(sta_B[4]);B_50.append(sta_B[5]);B_75.append(sta_B[6]);B_max.append(sta_B[7]);
    G_mean.append(sta_G[1]);G_std.append(sta_G[2]);G_min.append(sta_G[3]);G_25.append(sta_G[4]);G_50.append(sta_G[5]);G_75.append(sta_G[6]);G_max.append(sta_G[7]);
    R_mean.append(sta_R[1]);R_std.append(sta_R[2]);R_min.append(sta_R[3]);R_25.append(sta_R[4]);R_50.append(sta_R[5]);R_75.append(sta_R[6]);R_max.append(sta_R[7]);
    RE_mean.append(sta_RE[1]);RE_std.append(sta_RE[2]);RE_min.append(sta_RE[3]);RE_25.append(sta_RE[4]);RE_50.append(sta_RE[5]);RE_75.append(sta_RE[6]);RE_max.append(sta_RE[7]);
    NIR_mean.append(sta_NIR[1]);NIR_std.append(sta_NIR[2]);NIR_min.append(sta_NIR[3]);NIR_25.append(sta_NIR[4]);NIR_50.append(sta_NIR[5]);NIR_75.append(sta_NIR[6]);NIR_max.append(sta_NIR[7]);
    T_mean.append(sta_Tr[1]);T_std.append(sta_Tr[2]);T_min.append(sta_Tr[3]);T_25.append(sta_Tr[4]);T_50.append(sta_Tr[5]);T_75.append(sta_Tr[6]);T_max.append(sta_Tr[7]);
    NDVI_mean.append(sta_NDVI[1]);NDVI_std.append(sta_NDVI[2]);NDVI_min.append(sta_NDVI[3]);NDVI_25.append(sta_NDVI[4]);NDVI_50.append(sta_NDVI[5]);NDVI_75.append(sta_NDVI[6]);NDVI_max.append(sta_NDVI[7]);
    GNDVI_mean.append(sta_GNDVI[1]);GNDVI_std.append(sta_GNDVI[2]);GNDVI_min.append(sta_GNDVI[3]);GNDVI_25.append(sta_GNDVI[4]);GNDVI_50.append(sta_GNDVI[5]);GNDVI_75.append(sta_GNDVI[6]);GNDVI_max.append(sta_GNDVI[7]);
    RENDVI_mean.append(sta_RENDVI[1]);RENDVI_std.append(sta_RENDVI[2]);RENDVI_min.append(sta_RENDVI[3]);RENDVI_25.append(sta_RENDVI[4]);RENDVI_50.append(sta_RENDVI[5]);RENDVI_75.append(sta_RENDVI[6]);RENDVI_max.append(sta_RENDVI[7]);
    print("Processed:",str(ishp+1),'/',num_records)

end = datetime.datetime.now()
elapsed = end - start
print(elapsed.seconds,":",elapsed.microseconds) 

Processed: 1 / 213
Processed: 2 / 213
Processed: 3 / 213
Processed: 4 / 213
Processed: 5 / 213
Processed: 6 / 213
Processed: 7 / 213
Processed: 8 / 213
Processed: 9 / 213
Processed: 10 / 213
Processed: 11 / 213
Processed: 12 / 213
Processed: 13 / 213
Processed: 14 / 213
Processed: 15 / 213
Processed: 16 / 213
Processed: 17 / 213
Processed: 18 / 213
Processed: 19 / 213
Processed: 20 / 213
Processed: 21 / 213
Processed: 22 / 213
Processed: 23 / 213
Processed: 24 / 213
Processed: 25 / 213
Processed: 26 / 213
Processed: 27 / 213
Processed: 28 / 213
Processed: 29 / 213
Processed: 30 / 213
Processed: 31 / 213
Processed: 32 / 213
Processed: 33 / 213
Processed: 34 / 213
Processed: 35 / 213
Processed: 36 / 213
Processed: 37 / 213
Processed: 38 / 213


In [6]:
df_result = pd.DataFrame()
df_result['ID'] = ID
df_result['B_mean'] = B_mean
df_result['B_std'] = B_std
df_result['B_min'] = B_min
df_result['B_25'] = B_25
df_result['B_50'] = B_50
df_result['B_75'] = B_75
df_result['B_max'] = B_max
df_result['G_mean'] = G_mean
df_result['G_std'] = G_std
df_result['G_min'] = G_min
df_result['G_25'] = G_25
df_result['G_50'] = G_50
df_result['G_75'] = G_75
df_result['G_max'] = G_max
df_result['R_mean'] = R_mean
df_result['R_std'] = R_std
df_result['R_min'] = R_min
df_result['R_25'] = R_25
df_result['R_50'] = R_50
df_result['R_75'] = R_75
df_result['R_max'] = R_max
df_result['RE_mean'] = RE_mean
df_result['RE_std'] = RE_std
df_result['RE_min'] = RE_min
df_result['RE_25'] = RE_25
df_result['RE_50'] = RE_50
df_result['RE_75'] = RE_75
df_result['RE_max'] = RE_max
df_result['NIR_mean'] = NIR_mean
df_result['NIR_std'] = NIR_std
df_result['NIR_min'] = NIR_min
df_result['NIR_25'] = NIR_25
df_result['NIR_50'] = NIR_50
df_result['NIR_75'] = NIR_75
df_result['NIR_max'] = NIR_max
df_result['T_mean'] = T_mean
df_result['T_std'] = T_std
df_result['T_min'] = T_min
df_result['T_25'] = T_25
df_result['T_50'] = T_50
df_result['T_75'] = T_75
df_result['T_max'] = T_max
df_result['NDVI_mean'] = NDVI_mean
df_result['NDVI_std'] = NDVI_std
df_result['NDVI_min'] = NDVI_min
df_result['NDVI_25'] = NDVI_25
df_result['NDVI_50'] = NDVI_50
df_result['NDVI_75'] = NDVI_75
df_result['NDVI_max'] = NDVI_max
df_result['GNDVI_mean'] = GNDVI_mean
df_result['GNDVI_std'] = GNDVI_std
df_result['GNDVI_min'] = GNDVI_min
df_result['GNDVI_25'] = GNDVI_25
df_result['GNDVI_50'] = GNDVI_50
df_result['GNDVI_75'] = GNDVI_75
df_result['GNDVI_max'] = GNDVI_max
df_result['RENDVI_mean'] = RENDVI_mean
df_result['RENDVI_std'] = RENDVI_std
df_result['RENDVI_min'] = RENDVI_min
df_result['RENDVI_25'] = RENDVI_25
df_result['RENDVI_50'] = RENDVI_50
df_result['RENDVI_75'] = RENDVI_75
df_result['RENDVI_max'] = RENDVI_max


In [7]:
df_result

,ID,B_mean,B_std,B_min,B_25,B_50,B_75,B_max,G_mean,G_std,...,GNDVI_50,GNDVI_75,GNDVI_max,RENDVI_mean,RENDVI_std,RENDVI_min,RENDVI_25,RENDVI_50,RENDVI_75,RENDVI_max
0,1,0.025806,0.009148,0.012024,0.018089,0.023941,0.030556,0.053467,0.055107,0.012466,...,0.742529,0.784695,0.834503,0.532044,0.088900,0.276097,0.484533,0.548507,0.597910,0.676346
1,2,0.025739,0.007982,0.011017,0.019730,0.024689,0.031166,0.044769,0.051311,0.012163,...,0.724662,0.776689,0.828192,0.522224,0.095424,0.244786,0.472918,0.522149,0.598698,0.692254
2,3,0.035256,0.016694,0.008606,0.020599,0.031754,0.048859,0.071869,0.062831,0.019166,...,0.687951,0.768826,0.839572,0.458316,0.137922,0.053167,0.358696,0.479891,0.570350,0.703690
3,4,0.031295,0.013380,0.011292,0.020180,0.029099,0.042671,0.060211,0.057350,0.016571,...,0.702967,0.761252,0.842939,0.491854,0.116199,0.211743,0.401489,0.503873,0.580683,0.708857
4,5,0.025845,0.009208,0.010468,0.019676,0.023651,0.031456,0.055908,0.053705,0.011292,...,0.752796,0.785741,0.838269,0.534444,0.107230,0.212753,0.482558,0.563758,0.612175,0.703117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,181,0.025074,0.011806,0.010712,0.015808,0.020477,0.033134,0.053375,0.055151,0.017644,...,0.768467,0.808461,0.856040,0.545811,0.111412,0.250109,0.466715,0.575809,0.630477,0.717922
181,182,0.022396,0.007475,0.010254,0.016716,0.020554,0.026093,0.052094,0.056746,0.012675,...,0.789136,0.811097,0.849673,0.599226,0.052266,0.374901,0.565033,0.605967,0.640230,0.696436
182,183,0.039189,0.016552,0.012024,0.026253,0.038101,0.048721,0.081940,0.073756,0.023136,...,0.639863,0.698079,0.818821,0.406568,0.112480,0.190975,0.326245,0.408460,0.472695,0.650269
183,184,0.029053,0.012698,0.008667,0.017845,0.027283,0.035400,0.062836,0.058544,0.019324,...,0.750975,0.799559,0.877650,0.544495,0.112127,0.220747,0.467956,0.568559,0.630929,0.733960


In [8]:
df_result.to_csv(dir_tmp+'\\072X.csv')